# Excel Extract: Book 4

## Extract files using file depths

In [1]:
import glob
import os
import pandas as pd
import re
import jellyfish
import numpy as np
import itertools
import seaborn as sns
sns.set()
from collections import Counter

In [2]:
from inspect import getouterframes, currentframe
import os

def runrec(src):
    '''
    Modified function from link above
    to yield tuples where the [1] index
    of each tuple is an int representing
    the depth of that dir or file in 
    the system.
    '''
    level = len(getouterframes(currentframe()))
    yield (src, level)
    for x in os.listdir(src):
        srcname = os.path.join(src, x)
        if os.path.isdir(srcname):
            yield from runrec(srcname)
        else:
            yield (srcname,level+1)

a = list(runrec(r"Z:\Data\Surveys\Quarterly Mutual Fund Survey\2018") )

def groupings(list_of_tuples):
    '''
    Takes in a list_of_tuples, where the 
    [0] index is a path name and the [1]
    index is the depth. Returns a dict
    where the keys are the depths and 
    the paths are a list assigned to the
    keys.
    '''
    groups = {}
    for i in list_of_tuples:
        if i[1] in groups.keys():
            groups[i[1]].append(i[0]) # groups = {i[k]:['path1','path2'...], ...}
        else:
            groups[i[1]] = [i[0]]
        
    return groups

d = groupings(a)

def largest_group(dict_of_lists):
    '''
    Takes in the above output and
    finds largest group. Outputs
    group as a list.
    '''
    values = list(map(lambda x: len(dict_of_lists[x]), dict_of_lists.keys()))
    index_max = max(range(len(values)), key=values.__getitem__)
    key_with_most_paths = list(dict_of_lists.keys())[index_max]
    return dict_of_lists[key_with_most_paths]

#Right now this function naively finds the largest grouping of files at a given depth in the file tree. A ui version would suggest groupings of files to the user, by descending group size. 

list_of_files = largest_group(d) # stored in hardcoded2

## Scanning and Scraping Columns from the file list 

The problem is that surveys often are not formatted identically. One option is to hardcode a set of rules that fail if the survey deviates. A better solution might be to loop through the files, locate a target sheet, and locate target columns. In a ui, the user might supply the target sheet and target columns. The loop could ask for clarigying input when it finds targets too disimilar to the supplied objective.

### Road map:
1. **Sheet Process**
    - User input sheet name objective OR group similar targets and suggest
    - generate list_of_sheets from list_of_files
    - check list_of_sheets for targets that match objective, and request clarification if there are gaps
<br>
<br>
2. **Column Process**
    - User input columns objective
    - check target sheet for columns objective and if found, organize and append data to master table

### 1. Sheet Process 

Let's explore if we can group sheet names:

In [3]:
def list_of_sheets(list_of_files):
    '''
    '''
    dict_of_sheets = {}
    
    common_substring = os.path.commonprefix(list_of_files)
    
    for file in list_of_files:
        try:
            dict_of_sheets[file.split(common_substring)[1]] = list(pd.read_excel(file, sheet_name = None).keys())
        except:
            print('The following file could not be read as an excel: {}'.format(file))
    
    return dict_of_sheets

Upon reflection, a better process might be to generate a flat list of sheet names, recommend one of the top ones and use that as the sheet objective.

In [4]:
def sheet_name_freq(list_of_files):
    '''
    '''
    sheet_freq = []
    
    for file in list_of_files:
        try:
            sheets = list(pd.read_excel(file, sheet_name = None).keys())
        except:
            print('\nThe following file could not be read as an excel: {}'.format(file))
        
        sheet_freq += sheets
    
    return Counter(sheet_freq)
    

Choose one the most frequent sheet names, eg. Load Survey, and then check which files have a sheet which can be loosely matched to this objective.

In [5]:
sheet_objective = 'Load Survey'

In [6]:
# list of list output for POC for matching_sheet_objective()
def sheet_name_freq_list(list_of_files):
    '''
    '''
    sheet_freq = []
    
    for file in list_of_files:
        try:
            sheets = list(pd.read_excel(file, sheet_name = None).keys())
            sheet_freq.append(sheets)
        except:
            print('\nThe following file could not be read as an excel: {}'.format(file))
    
    return sheet_freq

In [7]:
def matching_sheet_objective(objective, list_of_files):
    '''
    '''
    dict_of_df = {}
    
    common_substring = os.path.commonprefix(list_of_files)
    
    for file in list_of_files:
        
        try:
            curr_df = pd.read_excel(file, sheet_name = None)
        except:
            print('\nThe following file could not be read as an excel: {}'.format(file))
    
        curr_sheets = list(curr_df.keys())
        
        if objective in curr_sheets:
            dict_of_df[file.split(common_substring)[1]] = curr_df[objective]
        
        else:
            
            print('No targets match objective. Prompt user for input') # prompt user for guided input
            '''
            print('\nObjective sheet: "{}" is not in list of sheet targets: ')
            
            for i in range(len(curr_sheets)):
                print('{}. {}'.format(i,curr_sheets[i]))
            
            choice = input('Enter the number of the correct above target. Enter nothing to skip: ')
            
            if choice == '':
                print('Skipped. No sheets from the following file were imported: {}'.format(choice,file))
                
            elif int(choice) in range(len(curr_sheets)):
                dict_of_df[file.split(common_substring)[1]] = curr_df[curr_sheets[int(choice)]]
                print('{} was imported.'.format(curr_sheets[int(choice)]))
            
            else:
                print('"{}" is not a valid input. No sheets from the following file were imported: {}'.format(choice,file))
            '''
    return dict_of_df

In [8]:
matchedSheets = matching_sheet_objective(sheet_objective,list_of_files)

No targets match objective. Prompt user for input
No targets match objective. Prompt user for input

The following file could not be read as an excel: Z:\Data\Surveys\Quarterly Mutual Fund Survey\2018\Q1\Received\AVISO\RE Quarterly mutual fund survey - provincial data inquiries.msg
No targets match objective. Prompt user for input
No targets match objective. Prompt user for input
No targets match objective. Prompt user for input
No targets match objective. Prompt user for input
No targets match objective. Prompt user for input
No targets match objective. Prompt user for input
No targets match objective. Prompt user for input

The following file could not be read as an excel: Z:\Data\Surveys\Quarterly Mutual Fund Survey\2018\Q1\Received\Manulife\RE Quarterly mutual fund survey - provincial data inquiries.msg
No targets match objective. Prompt user for input
No targets match objective. Prompt user for input

The following file could not be read as an excel: Z:\Data\Surveys\Quarterly Mutu

In [9]:
from IPython.display import display, HTML
keyIndex = list(matchedSheets.keys())
for i in range(10):
    display(matchedSheets[keyIndex[i]])

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Load Funds Survey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Company:,AGF Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Do not include Institutional series.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"For back-end, front-end and low load please in...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,All Funds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Date,Load structures,Assets,Gross sales,Redemptions,NaN,NaN,NaN,NaN,NaN


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Load Funds Survey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Company:,NEI Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Do not include Institutional series.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"For back-end, front-end and low load please in...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,All Funds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Date,Load structures,Assets,Gross sales,Redemptions,NaN,NaN,NaN,NaN,NaN


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Load Funds Survey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Company:,NEI Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Do not include Institutional series.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"For back-end, front-end and low load please in...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,All Funds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Date,Load structures,Assets,Gross sales,Redemptions,NaN,NaN,NaN,NaN,NaN


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Load Funds Survey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Company:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Do not include Institutional series.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"For back-end, front-end and low load please in...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,All Funds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Date,Load structures,Assets,Gross sales,Redemptions,NaN,NaN,NaN,NaN,NaN


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Load Funds Survey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Company:,Bridgehouse Asset Managers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Do not include Institutional series.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"For back-end, front-end and low load please in...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,All Funds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Date,Load structures,Assets,Gross sales,Redemptions,NaN,NaN,NaN,NaN,NaN


,Load Funds Survey,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,Company:,CIBC MUTUAL FUNDS,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,"For back-end, front-end and low load please in...",NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,All Funds,NaN,NaN,NaN,NaN
6,Date,Load structures,Assets,Gross sales,Redemptions
7,Q1 2018,Back-end load,0,0,0
8,NaN,Front-end load,0,0,0
9,NaN,Low load,0,0,0


,Load Funds Survey,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Company:,Renaissance Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"For back-end, front-end and low load please in...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,All Funds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Date,Load structures,Assets,Gross sales,Redemptions,NaN,NaN,NaN,NaN
7,Q1 2018,Back-end load,9.18689e+08,1.07172e+07,4.04611e+07,NaN,0.052448,0.008389,0.034936
8,NaN,Front-end load,5904414548,2.15111e+08,4.02674e+08,NaN,0.337083,0.168382,0.347692
9,NaN,Low load,1295241683,2.42273e+07,7.09142e+07,NaN,0.073945,0.018964,0.061231


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Load Funds Survey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Company:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Do not include Institutional series.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"For back-end, front-end and low load please in...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,All Funds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Date,Load structures,Assets,Gross sales,Redemptions,NaN,NaN,NaN,NaN,NaN


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Load Funds Survey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Company:,Dynamic Funds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Do not include Institutional series.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"For back-end, front-end and low load please in...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,All Funds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Date,Load structures,Assets,Gross sales,Redemptions,NaN,NaN,NaN,NaN,NaN


,IE - Load Funds Survey (Quarterly),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Company : FIDELITY INVESTMENTS CANADA ULC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Period : Q1 2018 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Quarter Ending,Load Structures,Assets,Gross Sales,Redemptions,Avg Load Charged in Front-load Option,NaN,NaN,NaN,NaN,NaN
5,Q1 2018,Back-end Load Option,9.37522e+09,3.23329e+08,-2.08628e+08,NaN,NaN,NaN,0.095566,0.061745,0.039100
6,Q1 2018,Front-end Load Option,5.84199e+10,2.5277e+09,-3.01033e+09,0.0%,NaN,NaN,0.595499,0.482708,0.564177
7,Q1 2018,Low Load Option,4.65393e+09,2.33082e+08,-1.67146e+08,NaN,NaN,NaN,0.047440,0.044511,0.031325
8,Q1 2018,No-Load Option,2.56534e+10,2.15239e+09,-1.94968e+09,NaN,NaN,NaN,0.261496,0.411036,0.365397
9,NaN,Sum:,9.81025e+10,5236499135,-5.33579e+09,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
matchedSheets[list(matchedSheets.keys())[1]]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Load Funds Survey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Company:,NEI Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Do not include Institutional series.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"For back-end, front-end and low load please in...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,All Funds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Date,Load structures,Assets,Gross sales,Redemptions,NaN,NaN,NaN,NaN,NaN


In [11]:
matchedSheets[list(matchedSheets.keys())[1]]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Load Funds Survey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Company:,NEI Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Do not include Institutional series.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"For back-end, front-end and low load please in...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,All Funds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Date,Load structures,Assets,Gross sales,Redemptions,NaN,NaN,NaN,NaN,NaN


### 2. Column Process

Using the dict of pandas dataframe objects in matchedSheets, develop an algorithm to find column header targets. Ideally, a list of objective targets could be found and suggested. The key for each dataframe will have to be parsed to provide information about the frame and to populate some columns (eg. Sponsor).

In [13]:
chr(ord('A') + 2)

'C'